In [1]:
#Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pyautogui
import random
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [2]:
#Funciones
#mover el mouse y hacer click
def move_mouse_random_straight_line(target_x, target_y):
    current_x, current_y = pyautogui.position()

    deviation_range = 2

    # Calculate the distance between current and target position
    distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

    # Number of steps to take in the straight line trajectory
    num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
    if num_steps == 0:
        num_steps = 1  # Ajusta el tamaño del paso según sea necesario

    # Calculate step size in x and y directions
    step_x = (target_x - current_x) / num_steps
    step_y = (target_y - current_y) / num_steps

    # To stop the process when is close enough
    stop_process = 0

    # Move the mouse in a straight line trajectory
    while current_x != target_x or current_y != target_y:
        current_x, current_y = pyautogui.position()

        # Calculate the distance between current and target position
        distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5

        # Number of steps to take in the straight line trajectory
        num_steps = int(distance / 20)  # Ajusta el tamaño del paso según sea necesario
        if num_steps == 0:
            # To stop the process when is close enough
            stop_process = stop_process + 1
            num_steps = 1  # Ajusta el tamaño del paso según sea necesario
            deviation_range = 1

        # Calculate step size in x and y directions
        step_x = (target_x - current_x) / num_steps
        step_y = (target_y - current_y) / num_steps

        current_x += step_x + random.randint(-deviation_range, deviation_range)
        current_y += step_y + random.randint(-deviation_range, deviation_range)
        pyautogui.moveTo(current_x, current_y, duration=0.01)
def move_mouse_and_click(x_pos, y_pos, input_string=False):
    move_mouse_random_straight_line(x_pos, y_pos)
    time.sleep(random.random())
    pyautogui.press("enter")
    
    # Perform a mouse click at the target location
    pyautogui.click(x_pos, y_pos)

# Función para recorrer del 16 al 25
def recorrer_rango_16_25(driver, csv_writer):
    for numero in range(16, 26):
        procesar_categoria(driver, csv_writer, numero)

# Función para recorrer del 26 al 31
def recorrer_rango_26_31(driver, csv_writer):
    for numero in range(26, 32):
        procesar_categoria(driver, csv_writer, numero)

# Función para procesar una categoría específica
def procesar_categoria(driver, csv_writer, numero):
    identificador_categoria = f"nav_em_1_1_1_{numero}"

    # Buscar el enlace de la categoría utilizando el identificador
    enlace_categoria = driver.find_element(By.XPATH, f"//a[@class='hmenu-item' and contains(@data-ref-tag, '{identificador_categoria}')]")
    
    # Haz clic en la categoría
    enlace_categoria.click()
    
    # Espera 2 segundos (opcional, ajusta según tus necesidades)
    time.sleep(2)

    # Encontrar la lista de subcategorías
    lista_subcategorias = driver.find_elements(By.XPATH, "//ul[@class='hmenu hmenu-visible hmenu-translateX']/li/a[@class='hmenu-item']")

    # Crear un diccionario para almacenar las subcategorías y sus enlaces
    subcategorias = {}

    # Iterar sobre las subcategorías y almacenar en el diccionario
    for subcategoria in lista_subcategorias:
        nombre_subcategoria = subcategoria.text
        enlace_subcategoria = subcategoria.get_attribute("href")
        subcategorias[nombre_subcategoria] = enlace_subcategoria

    # Imprimir el diccionario (puedes adaptar esta parte según tus necesidades)
    for nombre, enlace in subcategorias.items():
        print(f"{nombre}: {enlace}")

        # Escribir la fila en el archivo CSV
        csv_writer.writerow([f"Categoría {numero}", nombre, enlace])

    # Volver atrás para la siguiente iteración
    move_mouse_and_click(47, 146)

    # Espera 2 segundos (opcional, ajusta según tus necesidades)
    time.sleep(2)

#scrapea los links, por los numeros de paginas que le damos en enlaces_final.csv, configurar el numero de paginas requeridas en num_pages, 
def scrape_amazon_products_from_csv(csv_file, num_pages=1, wait_time=3, output_file='resultados_amazon.csv'):
    resultados_totales = []

    # Configurar Selenium para Firefox
    options = webdriver.FirefoxOptions()
    options.headless = True  # Comenta esta linea si queres ver el proceso en el navegador
    driver = webdriver.Firefox(options=options)
    # Maximizar la ventana del navegador
    driver.maximize_window()

    def click_ver_mas_resultados(driver):
        try:
            # Esperar a que el enlace "Ver todos los resultados" sea visible
            ver_mas_resultados = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//a[@id="apb-desktop-browse-search-see-all"]'))
            )

            # Obtener la URL del enlace
            url_ver_todos_resultados = ver_mas_resultados.get_attribute('href')

            # Hacer clic en el enlace
            ver_mas_resultados.click()
            print(f"Hizo clic en 'Ver todos los resultados' con éxito. URL: {url_ver_todos_resultados}")

        except Exception as e:
            print(f"Error al hacer clic en 'Ver todos los resultados': {str(e)}")

    def extraer_info_productos(driver):
        try:
            # Esperar a que los elementos se carguen
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 's-main-slot'))
            )

            # Obtener el HTML de la página actual
            html = driver.page_source

            # Utilizar BeautifulSoup para analizar el HTML
            soup = BeautifulSoup(html, 'html.parser')

            # Buscar directamente las etiquetas div con el atributo data-asin
            productos = soup.find_all('div', {'data-asin': True})

            resultados = []

            for producto in productos:
                asin = producto['data-asin']

                titulo_element = producto.find('span', class_='a-text-normal')
                titulo = titulo_element.text.strip() if titulo_element else 'No disponible'

                enlace_element = producto.find('a', class_='a-link-normal')
                enlace = enlace_element['href'] if enlace_element else 'No disponible'

                precio_element = producto.find('span', class_='a-price')
                precio = precio_element.find('span', class_='a-offscreen').text.strip() if precio_element else 'No disponible'

                resultados.append({
                    'asin': asin,
                    'titulo': titulo,
                    'enlace': f"https://www.amazon.com.mx{enlace}",
                    'precio': precio
                    # Puedes agregar más campos según tus necesidades
                })

            return resultados

        except Exception as e:
            print(f"Error: {e}")
            return None

    with open(csv_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            categoria = row['Subcategoría']
            url = row['Enlace']

            print(f"Navegando en la categoría: {categoria}")
            categoria_resultados = {'Categoria': categoria, 'Productos': []}

            try:
                # Navegar a la página actual
                driver.get(url)

                # Llamar a la función click_ver_mas_resultados solo la primera vez
                click_ver_mas_resultados(driver)

                for page_num in range(1, num_pages + 1):
                    print(f"Navegando a la página {page_num}")

                    # Esperar a que la página se cargue completamente
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-asin]'))
                    )
                    WebDriverWait(driver, 10).until(
                        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div[data-asin]'))
                    )

                    # Llamar a la función extraer_info_productos para obtener los productos
                    productos_extraidos = extraer_info_productos(driver)

                    # Almacenar resultados en la lista total
                    if productos_extraidos:
                        categoria_resultados['Productos'].extend(productos_extraidos)

                    # Esperar el tiempo especificado entre páginas
                    time.sleep(wait_time)

            except Exception as e:
                print(f"Error en la categoría {categoria}: {e}")

            # Almacenar resultados de la categoría en la lista total
            if categoria_resultados['Productos']:
                resultados_totales.append(categoria_resultados)

    # Cerrar el driver al finalizar
    #driver.quit()

    # Guardar resultados en un archivo CSV con las columnas especificadas
    if resultados_totales:
        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Categoria', 'Titulo', 'Enlace', 'Precio']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            for categoria_resultados in resultados_totales:
                for producto in categoria_resultados['Productos']:
                    writer.writerow({
                        'Categoria': categoria_resultados['Categoria'],
                        'Titulo': producto['titulo'],
                        'Enlace': producto['enlace'],
                        'Precio': producto['precio']
                    })

#obtener el nombre del vendedor
def obtener_vendedor(url):
    options = webdriver.FirefoxOptions()
    options.headless = True
    driver = webdriver.Firefox(options=options)

    try:
        driver.get(url)
        vendedor_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'offer-display-feature-text-message'))
        )
        vendedor = vendedor_element.text.strip()
        return vendedor
    except TimeoutException:
        print(f"Tiempo de espera agotado para {url}")
        return 'No encontrado'
    except NoSuchElementException:
        print(f"No se encontró el elemento del vendedor para {url}")
        return 'No encontrado'
    except Exception as e:
        print(f"Error al obtener el vendedor para {url}: {e}")
        return 'No encontrado'
    finally:
        driver.quit()

In [ ]:
# Configurar Selenium para Firefox
options = webdriver.FirefoxOptions()
# options.headless = True  # Descomenta esta línea si deseas ejecutar Firefox en modo headless
driver = webdriver.Firefox(options=options)
# Maximizar la ventana del navegador
driver.maximize_window()
url2 = 'https://www.amazon.com.mx/ref=nav_bb_logo'
driver.get(url2)

# Espera 5 segundos (ajustado a tu código)
time.sleep(5)

while True:
    try:
        # Intenta hacer clic en el botón "Ver todo"
        ver_todo_button = driver.find_element(By.XPATH, "//a[contains(@class, 'hmenu-compressed-btn') and contains(div, 'Ver todo')]")
        ver_todo_button.click()
        break  # Sale del bucle si encuentra y hace clic en el botón "Ver todo"
    except NoSuchElementException:
        # Si no encuentra el botón "Ver todo", mueve el mouse y haz clic
        move_mouse_and_click(66, 145)
        time.sleep(1)  # Ajusta el tiempo de espera según sea necesario


# Función para recorrer del 16 al 25
def recorrer_rango_16_25(driver, csv_writer):
    for numero in range(16, 26):
        procesar_categoria(driver, csv_writer, numero)

# Función para recorrer del 26 al 31
def recorrer_rango_26_31(driver, csv_writer):
    for numero in range(26, 32):
        procesar_categoria(driver, csv_writer, numero)

# Función para procesar una categoría específica
def procesar_categoria(driver, csv_writer, numero):
    identificador_categoria = f"nav_em_1_1_1_{numero}"

    # Buscar el enlace de la categoría utilizando el identificador
    enlace_categoria = driver.find_element(By.XPATH, f"//a[@class='hmenu-item' and contains(@data-ref-tag, '{identificador_categoria}')]")
    
    # Haz clic en la categoría
    enlace_categoria.click()
    
    # Espera 2 segundos (opcional, ajusta según tus necesidades)
    time.sleep(2)

    # Encontrar la lista de subcategorías
    lista_subcategorias = driver.find_elements(By.XPATH, "//ul[@class='hmenu hmenu-visible hmenu-translateX']/li/a[@class='hmenu-item']")

    # Crear un diccionario para almacenar las subcategorías y sus enlaces
    subcategorias = {}

    # Iterar sobre las subcategorías y almacenar en el diccionario
    for subcategoria in lista_subcategorias:
        nombre_subcategoria = subcategoria.text
        enlace_subcategoria = subcategoria.get_attribute("href")
        subcategorias[nombre_subcategoria] = enlace_subcategoria

    # Imprimir el diccionario (puedes adaptar esta parte según tus necesidades)
    for nombre, enlace in subcategorias.items():
        print(f"{nombre}: {enlace}")

        # Escribir la fila en el archivo CSV
        csv_writer.writerow([f"Categoría {numero}", nombre, enlace])

    # Volver atrás para la siguiente iteración
    move_mouse_and_click(47, 146)

    # Espera 2 segundos (opcional, ajusta según tus necesidades)
    time.sleep(2)

# Nombre del archivo CSV
csv_filename = 'enlaces_categorias_final.csv'

# Abrir el archivo CSV en modo de escritura
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    # Crear un objeto escritor CSV
    csv_writer = csv.writer(csvfile)

    # Escribir el encabezado del CSV
    csv_writer.writerow(['Categoría', 'Subcategoría', 'Enlace'])

    # Recorrer del 16 al 25
    recorrer_rango_16_25(driver, csv_writer)

    # Recorrer del 26 al 31
    recorrer_rango_26_31(driver, csv_writer)

In [ ]:
# Uso
csv_file_path = 'enlaces_categorias_final.csv'
scrape_amazon_products_from_csv(csv_file_path)


In [ ]:
def obtener_vendedor(url):
    options = webdriver.FirefoxOptions()
    options.headless = True
    driver = webdriver.Firefox(options=options)

    try:
        driver.get(url)
        # Esperar a que el elemento contenedor esté presente
        container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "averageCustomerReviews"))
        )
        
        # Obtener el HTML de la página web
        html = driver.page_source
        
        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        vendedor_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'sellerProfileTriggerId'))
        )
        vendedor = vendedor_element.text.strip()

        # Encontrar el elemento que contiene el puntaje promedio
        puntaje_element = soup.find('span', {'id': 'acrPopover'}).find('span', {'class': 'a-size-base'})
        puntaje_text = puntaje_element.text.strip()
        
        # Encontrar el elemento que contiene el número de calificaciones
        calificaciones_element = soup.find('span', {'id': 'acrCustomerReviewText'})
        calificaciones_text = calificaciones_element.text.strip()
        
        # Esperar a que el elemento contenedor esté presente
        container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "bylineInfo"))
        )
                
        # Encontrar el elemento que contiene la marca
        marca_element = soup.find('a', {'id': 'bylineInfo'})
        marca_text = marca_element.text.strip()

        return vendedor, marca_text, puntaje_text, calificaciones_text
    
    except TimeoutException:
        print(f"Tiempo de espera agotado para {url}")
        return 'No encontrado', '', 0.0, 0
    except NoSuchElementException:
        print(f"No se encontró el elemento del vendedor para {url}")
        return 'No encontrado', '', 0.0, 0
    except Exception as e:
        print(f"Error al obtener el vendedor para {url}: {e}")
        return 'No encontrado', '', 0.0, 0
    finally:
        driver.quit()


archivo_csv = 'resultados_amazon.csv'
archivo_csv_salida = 'amazon_final.csv'

try:
    with open(archivo_csv, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        fieldnames = reader.fieldnames + ['Vendedor', 'Marca', 'Puntuacion', 'Calificacion']
        rows = []
        for row in reader:
            url = row.get('Enlace')
            if url:
                vendedor, marca_text, puntaje_text, calificacion_text = obtener_vendedor(url)
                row['Vendedor'] = vendedor
                row['Marca'] = marca_text
                row['Puntuacion'] = puntaje_text
                row['Calificacion'] = calificacion_text
            else:
                print("No se encontró ningún enlace en la fila:", row)
                row['Vendedor'] = 'No encontrado'
                row['Marca'] = ''
                row['Puntuacion'] = ''
                row['Calificacion'] = ''
            rows.append(row)

    with open(archivo_csv_salida, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

except FileNotFoundError:
    print(f"No se pudo encontrar el archivo CSV: {archivo_csv}")
except Exception as e:
    print(f"Error al procesar el archivo CSV: {e}")
